# GREAT EXPECTATIONS FOR MOVIL BILLETERS

In [2]:
 # IMPORTACIONES
import numpy, scipy, pandas, sklearn, great_expectations as ge
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("Pandas:", pandas.__version__)
print("Sklearn:", sklearn.__version__)
print("GE:", ge.__version__)


NumPy: 1.26.4
SciPy: 1.10.1
Pandas: 2.2.2
Sklearn: 1.5.1
GE: 0.18.13


In [6]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA_PATH = Path("C:/Users/-zv/Documents/TRABAJO_MIADAS_MODULO7/proyecto-playstore/data/clean/comentarios_limpios_tokens.csv")

print("Usando:", DATA_PATH.resolve())

df_raw = pd.read_csv(DATA_PATH, encoding_errors="ignore")
print("RAW shape:", df_raw.shape)
df_raw.head(3)

Usando: C:\Users\-zv\Documents\TRABAJO_MIADAS_MODULO7\proyecto-playstore\data\clean\comentarios_limpios_tokens.csv
RAW shape: (1552, 14)


,userName,content,score,reviewCreatedVersion,fecha,hora,replyContent,repliedAt,appVersion,thumbsUpCount,content_limpio,content_limpio_corregido,content_normalizado,tokens
0,Ariel avillo Avillo acuña,la mejor aplicación para guardar mi dinero y p...,5,1.1.3,2025-09-27,12:53:46,Sin comentarios,NaN,1.1.3,0,mejor aplicación guardar dinero pagar,mejor aplicacion guardar dinero pagar,aplicacion guardar dinero pagar,"['aplicacion', 'guardar', 'dinero', 'pagar']"
1,Esther Sierra,no sirve quise crear mi cuenta y me sale error...,1,Desconocido,2025-09-27,12:49:35,Sin comentarios,NaN,Desconocido,0,sirve quise crear cuenta sale error red,sirle quite crear cuenta vale error red,sir él quite crear valer error red,"['sir', 'él', 'quite', 'crear', 'valer', 'erro..."
2,Marco Antonio Aguilar,es bien pero muy lenta la señal al pasar plata...,4,1.1.3,2025-09-27,12:41:05,Sin comentarios,NaN,1.1.3,0,bien lenta señal pasar plata comprobantes vece...,bien lenta senal pasar plata comprobante veras...,lento senal pasar plata comprobante vera llega...,"['lento', 'senal', 'pasar', 'plata', 'comproba..."


#####  Descripción de las columnas

Descripción de columnas

userName: Nombre o identificador del usuario que dejó la reseña.

content: Texto original de la reseña o comentario dejado por el usuario en la aplicación.

score: Puntuación otorgada por el usuario (por ejemplo, del 1 al 5).

categoria: Categoría asignada al score, transformando la puntuación en una etiqueta cualitativa (por ejemplo: "Muy bajo", "Bajo", "Medio", "Alto", "Muy alto").

reviewCreatedVersion: Versión de la aplicación en el momento en que se realizó la reseña.

fecha: Fecha en que el usuario dejó la reseña.

hora: Hora en que se realizó la reseña.

replyContent: Contenido de la respuesta oficial de la aplicación o desarrollador a la reseña del usuario (si existe).

repliedAt: Fecha y hora en que se respondió la reseña.

appVersion: Versión actual de la aplicación en la tienda o usada para esa reseña.

thumbsUpCount: Número de usuarios que marcaron la reseña como útil (“pulgar arriba”).

content_limpio: Texto de la reseña después de una limpieza inicial (eliminación de caracteres especiales, emojis, etc.).

content_limpio_corregido: Texto limpio tras corrección ortográfica y gramatical.

content_normalizado: Versión procesada del contenido, normalizada para análisis de texto (por ejemplo, minúsculas, eliminación de stopwords, reducción de diminutivos/superlativos).

tokens: Lista de palabras significativas extraídas del content_normalizado, útil para análisis de NLP.

app_id: Identificador único de la aplicación (por ejemplo: código interno o store ID).

# 1. Preprocesamiento minimos tipos

In [7]:
df_raw = df_raw.copy()

# Fecha → datetime
if "InvoiceDate" in df_raw.columns:
    df_raw["InvoiceDate"] = pd.to_datetime(df_raw["InvoiceDate"], errors="coerce")

# Números → asegurar tipo
for col in ["Quantity", "UnitPrice", "CustomerID"]:
    if col in df_raw.columns:
        df_raw[col] = pd.to_numeric(df_raw[col], errors="coerce")

print(df_raw.dtypes)
df_raw.isna().sum().sort_values(ascending=False).head(10)

userName                    object
content                     object
score                        int64
reviewCreatedVersion        object
fecha                       object
hora                        object
replyContent                object
repliedAt                   object
appVersion                  object
thumbsUpCount                int64
content_limpio              object
content_limpio_corregido    object
content_normalizado         object
tokens                      object
dtype: object


repliedAt                   1551
content_normalizado          176
content_limpio                 3
content_limpio_corregido       3
userName                       0
content                        0
score                          0
reviewCreatedVersion           0
fecha                          0
hora                           0
dtype: int64

# 2. Great Expectations Contexto y  datasource (idemponente)

In [8]:
import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.exceptions import DataContextError

# Contexto efímero en notebook
context = ge.get_context()

# Datasource runtime para Pandas (seguro si ya existe)
try:
    context.add_datasource(
        name="runtime_pandas_datasource",
        class_name="Datasource",
        execution_engine={"class_name": "PandasExecutionEngine"},
        data_connectors={
            "runtime_data_connector": {
                "class_name": "RuntimeDataConnector",
                "batch_identifiers": ["default_identifier_name"],
            }
        },
    )
except DataContextError:
    pass  # ya existe

# 3. VALIDACIÓN RAW (diagnóstico)
Objetivo: Chequeo médico inicial, confirma que el dataset es coherente en lo más básico, y que no hay problemas graves que impidan seguir adelante.

In [9]:
from great_expectations.core.expectation_suite import ExpectationSuite

# Batch request usando df_raw en memoria
batch_request_raw = RuntimeBatchRequest(
    datasource_name="runtime_pandas_datasource",
    data_connector_name="runtime_data_connector",
    data_asset_name="ecommerce_raw",
    runtime_parameters={"batch_data": df_raw},
    batch_identifiers={"default_identifier_name": "raw_run"},
)

# Crear/registrar suite RAW (0.18.x usa add_or_update)
suite_name_raw = "ecommerce_raw_suite"
try:
    suite_raw = context.get_expectation_suite(suite_name_raw)
except DataContextError:
    suite_raw = ExpectationSuite(expectation_suite_name=suite_name_raw)
    context.add_or_update_expectation_suite(expectation_suite=suite_raw)

# Obtener validator RAW
validator_raw = context.get_validator(
    batch_request=batch_request_raw,
    expectation_suite_name=suite_name_raw
)

# ---- EXPECTATIONS (menos estrictas, diagnóstico) ----
present = [c for c in ["InvoiceNo","StockCode","Description","Quantity",
                       "InvoiceDate","UnitPrice","CustomerID","Country"]
           if c in df_raw.columns]
validator_raw.expect_table_columns_to_match_set(present)

for c in ["InvoiceNo","StockCode","InvoiceDate","UnitPrice","Country"]:
    if c in df_raw.columns:
        validator_raw.expect_column_values_to_not_be_null(c, mostly=0.95)

if "UnitPrice" in df_raw.columns:
    validator_raw.expect_column_values_to_be_between("UnitPrice", min_value=0, max_value=None, mostly=0.99)
    #validator_raw.expect_column_values_to_be_greater_than_or_equal_to("UnitPrice", 0)

if "InvoiceDate" in df_raw.columns:
    validator_raw.expect_column_values_to_be_between(
        "InvoiceDate",
        min_value=pd.Timestamp("2010-12-01"),
        max_value=pd.Timestamp("2011-12-09"),
        parse_strings_as_datetimes=True,
        mostly=0.97
    )

if "InvoiceNo" in df_raw.columns:
    validator_raw.expect_column_values_to_match_regex("InvoiceNo", r"^[C]?\d+$", mostly=0.97)

if "Country" in df_raw.columns:
    top_countries = df_raw["Country"].dropna().value_counts().head(30).index.tolist()
    validator_raw.expect_column_values_to_be_in_set("Country", top_countries, mostly=0.98)

# Guardar suite y validar
validator_raw.save_expectation_suite(discard_failed_expectations=False)
res_raw = validator_raw.validate()
res_raw["statistics"]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{'evaluated_expectations': 1,
 'successful_expectations': 0,
 'unsuccessful_expectations': 1,
 'success_percent': 0.0}

In [14]:
print(res_raw)



{
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "expectation_type": "expect_table_columns_to_match_set",
        "kwargs": {
          "column_set": [],
          "batch_id": "53b10bbd6514577d42642464de031877"
        },
        "meta": {}
      },
      "result": {
        "observed_value": [
          "appVersion",
          "content",
          "content_limpio",
          "content_limpio_corregido",
          "content_normalizado",
          "fecha",
          "hora",
          "repliedAt",
          "replyContent",
          "reviewCreatedVersion",
          "score",
          "thumbsUpCount",
          "tokens",
          "userName"
        ],
        "details": {
          "mismatched": {
            "unexpected": [
              "appVersion",
              "content",
              "content_limpio",
              "content_limpio_corregido",
              "content_normalizado",
              "fecha",
              "hora",
  

#  4. Limpieza mínima
Este paso representó un filtro inicial para quedarnos con datos más confiables:
Quitamos errores evidentes.

Generamos una métrica clave (Revenue).
Comprobamos que el dataset mejoró, aunque una de las expectativas falló (lo que revela la presencia de casos atípicos que aún requieren análisis).
En resumen: el dataset pasara de ser “utilizable” a ser “más limpio y coherente”, aunque todavía no está perfecto.

In [10]:
df_clean = df_raw.copy()

# UnitPrice no negativo (devoluciones se reflejan en Quantity)
if "UnitPrice" in df_clean.columns:
    df_clean = df_clean[df_clean["UnitPrice"].fillna(0) >= 0]

# Quantity entero (si vino como float por nulos)
if "Quantity" in df_clean.columns and not np.issubdtype(df_clean["Quantity"].dtype, np.integer):
    df_clean["Quantity"] = pd.to_numeric(df_clean["Quantity"], errors="coerce").fillna(0).astype(int)

# Derivar Revenue
if set(["Quantity","UnitPrice"]).issubset(df_clean.columns):
    df_clean["Revenue"] = df_clean["Quantity"] * df_clean["UnitPrice"]

# Quitar duplicados exactos
df_clean = df_clean.drop_duplicates()

print("CLEAN shape:", df_clean.shape)
df_clean.head(3)

CLEAN shape: (1552, 14)


,userName,content,score,reviewCreatedVersion,fecha,hora,replyContent,repliedAt,appVersion,thumbsUpCount,content_limpio,content_limpio_corregido,content_normalizado,tokens
0,Ariel avillo Avillo acuña,la mejor aplicación para guardar mi dinero y p...,5,1.1.3,2025-09-27,12:53:46,Sin comentarios,NaN,1.1.3,0,mejor aplicación guardar dinero pagar,mejor aplicacion guardar dinero pagar,aplicacion guardar dinero pagar,"['aplicacion', 'guardar', 'dinero', 'pagar']"
1,Esther Sierra,no sirve quise crear mi cuenta y me sale error...,1,Desconocido,2025-09-27,12:49:35,Sin comentarios,NaN,Desconocido,0,sirve quise crear cuenta sale error red,sirle quite crear cuenta vale error red,sir él quite crear valer error red,"['sir', 'él', 'quite', 'crear', 'valer', 'erro..."
2,Marco Antonio Aguilar,es bien pero muy lenta la señal al pasar plata...,4,1.1.3,2025-09-27,12:41:05,Sin comentarios,NaN,1.1.3,0,bien lenta señal pasar plata comprobantes vece...,bien lenta senal pasar plata comprobante veras...,lento senal pasar plata comprobante vera llega...,"['lento', 'senal', 'pasar', 'plata', 'comproba..."


#  5. VALIDACIÓN CLEAN (más estricta)

In [11]:
# Batch request usando df_clean en memoria
batch_request_clean = RuntimeBatchRequest(
    datasource_name="runtime_pandas_datasource",
    data_connector_name="runtime_data_connector",
    data_asset_name="ecommerce_clean",
    runtime_parameters={"batch_data": df_clean},
    batch_identifiers={"default_identifier_name": "clean_run"},
)

# Crear/registrar suite CLEAN
suite_name_clean = "ecommerce_clean_suite"
try:
    suite_clean = context.get_expectation_suite(suite_name_clean)
except DataContextError:
    suite_clean = ExpectationSuite(expectation_suite_name=suite_name_clean)
    context.add_or_update_expectation_suite(expectation_suite=suite_clean)

# Validator CLEAN
validator_clean = context.get_validator(
    batch_request=batch_request_clean,
    expectation_suite_name=suite_name_clean
)

# ---- EXPECTATIONS (más estrictas) ----
expected_clean = [c for c in ["InvoiceNo","StockCode","Description","Quantity",
                              "InvoiceDate","UnitPrice","CustomerID","Country","Revenue"]
                  if c in df_clean.columns]
validator_clean.expect_table_columns_to_match_set(expected_clean)

for c in ["InvoiceNo","StockCode","InvoiceDate","UnitPrice","Country"]:
    if c in df_clean.columns:
        validator_clean.expect_column_values_to_not_be_null(c)

if "UnitPrice" in df_clean.columns:
    validator_clean.expect_column_values_to_be_between("UnitPrice", min_value=0, max_value=None)
    #validator_clean.expect_column_values_to_be_greater_than_or_equal_to("UnitPrice", 0)

if "Revenue" in df_clean.columns and df_clean["Revenue"].notna().sum() > 0:
    q01, q99 = df_clean["Revenue"].quantile([0.01, 0.99])
    validator_clean.expect_column_values_to_be_between("Revenue", min_value=q01, max_value=q99, mostly=0.98)

if "InvoiceNo" in df_clean.columns:
    validator_clean.expect_column_values_to_match_regex("InvoiceNo", r"^[C]?\\d+$")

validator_clean.save_expectation_suite(discard_failed_expectations=False)
res_clean = validator_clean.validate()
res_clean["statistics"]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{'evaluated_expectations': 1,
 'successful_expectations': 0,
 'unsuccessful_expectations': 1,
 'success_percent': 0.0}